In [1]:
import polars as pl
 
In_DF = pl.read_csv("Input M5/sales_train_evaluation.csv")

cal_df = pl.read_csv("Input M5/calendar.csv")

price_df = pl.read_csv("Input M5/sell_prices.csv")

In [2]:
all_col = [ col for col in In_DF.columns if not col.startswith("d_")]
day_cols = [ col for col in In_DF.columns if col.startswith("d_")]
d_cols_sorted = sorted(day_cols, key= lambda x: int(x.split('_')[1]))

df_last_365= In_DF.select(all_col + d_cols_sorted[-365:])

In [3]:
DCs=["CA_1","CA_2","CA_3","TX_1","TX_2"]

target_sample = 150

df_last_365 = (
    df_last_365.filter(pl.col("store_id").is_in(DCs)))

sku_count = (
    df_last_365.select(["dept_id", "cat_id", "item_id"])
    .unique()
    .group_by(["dept_id", "cat_id"])
    .agg(pl.col("item_id").n_unique().alias("Un_Sku_Count"))
)
total_sku = sku_count["Un_Sku_Count"].sum()

sku_count = sku_count.with_columns(
    ((pl.col("Un_Sku_Count")/total_sku) * target_sample)
    .round(0)
    .cast(int)
    .alias("sample_target")
)

sampled = []
for row in sku_count.iter_rows(named=True):
    subset = (
        df_last_365
        .filter((pl.col("dept_id") == row["dept_id"]) & (pl.col("cat_id")== row["cat_id"]))
        .select("item_id")
        .unique()
    )
    k = min(row["sample_target"], subset.height)
    sampled.append(subset.sample(n=k, seed=42))

sampled_iteams = pl.concat(sampled).unique()

fcst_df= (
df_last_365.join(sampled_iteams,on="item_id",how="right")
)

In [5]:
cal_df = cal_df.with_row_count("row")
cal_df = cal_df.with_columns(
    ("d_" + (pl.col("row") + 1).cast(str)).alias("d")
).drop("row")

cal_df = cal_df.select(["d", "date"])

d_columns = [ c for c in fcst_df.columns if c.startswith("d_")]

fcst_df_long = fcst_df.melt(
    id_vars=["item_id", "dept_id", "cat_id", "store_id"],
    value_vars= d_columns,
    variable_name="day",
    value_name="sales"
)

fcst_df_long = fcst_df_long.join(cal_df, right_on="d", left_on="day", how="left")

C:\Users\Przemek\AppData\Local\Temp\ipykernel_14536\2643017263.py:1: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  cal_df = cal_df.with_row_count("row")
C:\Users\Przemek\AppData\Local\Temp\ipykernel_14536\2643017263.py:10: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  fcst_df_long = fcst_df.melt(


In [6]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def forecast_sku_store(df_sku_store, forecast_horizon=45, seasonal_periods=7):

    ts = df_sku_store["sales"].values.astype(float)

    model = ExponentialSmoothing(
        ts,
        trend="add",
        seasonal="add",
        seasonal_periods=seasonal_periods
    )
    fit=model.fit()
    forecast = fit.forecast(forecast_horizon)
    return forecast
        

In [7]:
import datetime

fcst_df_long = fcst_df_long.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d")
)

maxday = fcst_df_long.select(pl.max("date")).item()

In [8]:
import pandas as pd 

fcst_pd = fcst_df_long.to_pandas()

forecast_horizon=45
seasonal_periods=7

forecast_list = []

for (item_id,store_id,cat_id,dept_id), group in fcst_pd.groupby(["item_id", "store_id", "cat_id", "dept_id" ]):
    fcst_values = forecast_sku_store(group, forecast_horizon, seasonal_periods)
    for i, val in enumerate(fcst_values, 1):
        forecast_list.append({
            "item_id":item_id,
            "store_id":store_id,
            "cat_id":cat_id,
            "dept_id":dept_id,
            "day": i,
            "forecast_sales": val
        })

fcst_o= pd.DataFrame(forecast_list)

fcst_o.loc[fcst_o['forecast_sales'] < 0, 'forecast_sales'] = 0

fcst_out=pl.from_pandas(fcst_o)

In [9]:
import datetime

fcst_out = fcst_out.with_columns((pl.lit(maxday, allow_object=True) + pl.duration(days=pl.col("day"))).alias("date")
                                )

In [10]:
pal_size_df = pl.DataFrame({
    "cat_id":["FOODS", "HOBBIES", "HOUSEHOLD"],
    "plant_id": ["PLANT_CA", "PLANT_TX", "PLANT_MX"],  
    "pallet_size":[ 4, 1, 2]
})

pal_size_df.write_csv("Symulation data/pal_size_df.csv") 

In [11]:
filt_price_df = price_df.group_by(["store_id","item_id"]).agg(pl.col("wm_yr_wk").max().alias("max_week"))
                                       
price_df_final = filt_price_df.join(price_df, left_on=["store_id","item_id","max_week"], right_on=["store_id","item_id", "wm_yr_wk"], how="left")

price_df_final.write_csv("Symulation data/price.csv")

In [12]:
import polars as pl

lane_rulles = [
    {"plant_id": "PLANT_CA", "cat_id": "FOODS", "store_id": "CA_1", "transit_time_days": 1},
    {"plant_id": "PLANT_CA", "cat_id": "FOODS", "store_id": "CA_2", "transit_time_days": 2},
    {"plant_id": "PLANT_CA", "cat_id": "FOODS", "store_id": "CA_3", "transit_time_days": 2},
    {"plant_id": "PLANT_CA", "cat_id": "FOODS", "store_id": "TX_1", "transit_time_days": 4},
    {"plant_id": "PLANT_CA", "cat_id": "FOODS", "store_id": "TX_2", "transit_time_days": 4},

    {"plant_id": "PLANT_TX", "cat_id": "HOBBIES", "store_id": "CA_1", "transit_time_days": 5},
    {"plant_id": "PLANT_TX", "cat_id": "HOBBIES", "store_id": "CA_2", "transit_time_days": 4},
    {"plant_id": "PLANT_TX", "cat_id": "HOBBIES", "store_id": "CA_3", "transit_time_days": 5},
    {"plant_id": "PLANT_TX", "cat_id": "HOBBIES", "store_id": "TX_1", "transit_time_days": 1},
    {"plant_id": "PLANT_TX", "cat_id": "HOBBIES", "store_id": "TX_2", "transit_time_days": 2},

    {"plant_id": "PLANT_MX", "cat_id": "HOUSEHOLD", "store_id": "CA_1", "transit_time_days": 3},
    {"plant_id": "PLANT_MX", "cat_id": "HOUSEHOLD", "store_id": "CA_2", "transit_time_days": 3},
    {"plant_id": "PLANT_MX", "cat_id": "HOUSEHOLD", "store_id": "CA_3", "transit_time_days": 3},
    {"plant_id": "PLANT_MX", "cat_id": "HOUSEHOLD", "store_id": "TX_1", "transit_time_days": 2},
    {"plant_id": "PLANT_MX", "cat_id": "HOUSEHOLD", "store_id": "TX_2", "transit_time_days": 2},
]

df_tt = pl.DataFrame(lane_rulles)
df_tt.write_csv("Symulation data/lane_rules.csv") 

**PROD TABLE**

In [13]:

plants = ["PLANT_CA", "PLANT_TX","PLANT_MX"]

fcst_out = (
    fcst_out.join(pal_size_df.select(["cat_id", "pallet_size"]), how="left", on="cat_id")
    .join(df_tt, how="left", on=["cat_id", "store_id"])
)

**BUILDING SAFETY STOCK**

In [14]:
import numpy as np

GRandGI= 1
PeriodBetweenReviews= 3

rng = np.random.default_rng(seed=42)

def random_deviation(n):
    return rng.uniform(0.8, 1.2, size=n)

n_rows = fcst_out.height

deviation = random_deviation(n_rows)

fcst_out = fcst_out.with_columns([
    pl.col("forecast_sales").mean().over(["item_id", "store_id"]).alias("avg_daily_demand")
])

fcst_out = fcst_out.with_columns([
    (pl.col("avg_daily_demand") * pl.Series(deviation)*(pl.col("transit_time_days")
    + GRandGI
    + PeriodBetweenReviews)
    ).round(0).alias("safety_stock")
])

demand_outlook = fcst_out.group_by(["item_id", "store_id"]).agg([
    pl.col("safety_stock").mean().alias("safety_stock_static")
]).join(fcst_out, on=["item_id", "store_id"], how="left")

buffer_days = np.array([5, 6, 7, 8, 9, 10, 11, 12])
prob = np.array([0.05, 0.10, 0.15, 0.15, 0.20, 0.20, 0.10, 0.05])

demand_outlook= demand_outlook.with_columns([    (pl.col("avg_daily_demand") *
     pl.Series(rng.choice(buffer_days, size=demand_outlook.height, p=prob))
    ).round(0).alias("SOG_initial")
])

demand_outlook = demand_outlook.sort(["item_id", "store_id", "day"])

demand_outlook = demand_outlook.with_columns([
    pl.when(pl.col("day") == 1)
    .then(pl.col("SOG_initial"))
    .otherwise(0)
    .alias("SOG_initial")
])

demand_outlook = demand_outlook.with_columns([
    (pl.col("SOG_initial").first() - pl.col("forecast_sales").cum_sum())
    .over(["item_id", "store_id"])
    .alias("projection")
])

demand_outlook = demand_outlook.with_columns([
    pl.when(pl.col("projection") < pl.col("safety_stock_static"))
    .then(pl.col("safety_stock_static") - pl.col("projection"))
    .otherwise(0)
    .alias("below_safety_stock_risk")
])

demand_outlook = demand_outlook.with_columns([
    pl.when(pl.col("projection") < 0)
    .then(-pl.col("projection")) 
    .otherwise(0)
    .alias("stock_out_risk")
])

demand_outlook = demand_outlook.with_columns([
    pl.col("below_safety_stock_risk").diff().over(["item_id", "store_id"]).alias("ss_risk_change"),
    pl.col("stock_out_risk").diff().over(["item_id", "store_id"]).alias("so_risk_change")
])

demand_outlook.write_csv("Symulation data/demand_outlook.csv")


In [15]:
min_date = demand_outlook.select(pl.col("date").min()).item()
min_day = demand_outlook.select(pl.col("day").min()).item()

demand_outlook = demand_outlook.with_columns([
    (pl.col("date") - pl.duration(days="transit_time_days")).alias("date"),
    (pl.col("day") - pl.col("transit_time_days")).alias("day")
])

demand_outlook = demand_outlook.with_columns([

    pl.when(pl.col("day") <= 0)
    .then(min_date)
    .otherwise(pl.col("date"))
    .alias("date"),
    
    pl.when(pl.col("day") <= 0)
    .then(1)  
    .otherwise(pl.col("day"))
    .alias("day")
])


In [16]:
Prod_req_df = demand_outlook.group_by(
    ["item_id", "plant_id", "store_id", "cat_id", "pallet_size", "transit_time_days", "date", "day"]).agg([
    pl.col("forecast_sales").sum().alias("daily_fcst"),
    pl.col("safety_stock_static").max().alias("safety_stock_static"),
    pl.col("below_safety_stock_risk").max().alias("current_ss_risk"),
    pl.col("stock_out_risk").max().alias("current_stockout_risk"),
    pl.col("ss_risk_change").max().alias("ss_risk_increase"),
    pl.col("so_risk_change").max().alias("so_risk_increase"),
    pl.col("projection").last().alias("current_projection"),
    pl.col("avg_daily_demand").sum().alias("avg_daily_demand")
])

Prod_req_df = Prod_req_df.with_columns([
    pl.when(pl.col("so_risk_increase") > 0)
    .then(pl.col("so_risk_increase"))
    .when(pl.col("ss_risk_increase") > 0)
    .then (pl.col("ss_risk_increase"))
    .otherwise(0)
    .alias("incremental_risk_requirement"),

    pl.when(pl.col("current_stockout_risk") > 0)
    .then(pl.lit("STOCKOUT"))
    .when(pl.col("current_ss_risk") > 0)
    .then(pl.lit("SS_RISK"))
    .otherwise(pl.lit(""))
    .alias("Urgency level")
])

Prod_req_df= Prod_req_df.with_columns([
    pl.col("incremental_risk_requirement").alias("prod_requirements")
])

Prod_req_df = Prod_req_df.sort(["plant_id", "item_id", "date"])

Prod_req_df = Prod_req_df.with_columns(
    pl.col("prod_requirements")
    .cum_sum()
    .over(["item_id", "plant_id"])
    .alias("cum_prod_requirements")
)

d_sog = np.array([0, 1, 2, 3, 4, 6, 7, 8,])
perc = np.array([0.12, 0.08, 0.16, 0.16, 0.14, 0.15, 0.14, 0.05])
random_effect = rng.uniform(0.90, 1.10, size=Prod_req_df.height)

Prod_req_df = Prod_req_df.with_columns([
    (pl.col("avg_daily_demand") *
    pl.Series(rng.choice(d_sog, p=perc, size=Prod_req_df.height))* pl.Series(random_effect)).alias("starting_inventory_plant")
])

Prod_req_df = Prod_req_df.with_columns([
    pl.when(pl.col("day") == 1)
    .then(pl.col("starting_inventory_plant"))
    .otherwise(0)
    .alias("starting_inventory_plant")
])

Prod_req_df.write_csv("Symulation data/Prod_req_df.csv")


In [17]:
rng = np.random.default_rng(seed=42)

cycles = np.array([2, 4, 5, 6, 8])
cycles_prod = {
    "PLANT_CA": np.array([0.25, 0.22, 0.27, 0.22, 0.04]),
    "PLANT_MX": np.array([0.32, 0.32, 0.22, 0.10, 0.04]),
    "PLANT_TX": np.array([0.32, 0.32, 0.16, 0.16, 0.04]),
}

## MOQ setup ##

p_challange = {
    "PLANT_CA": 0.15,
    "PLANT_MX": 0.10,
    "PLANT_TX": 0.05
}

def generate_prod_master(demand_outlook):
    avg_demand_plant = (
        demand_outlook
        .group_by(["item_id", "plant_id", "pallet_size"])
        .agg([
            pl.col("avg_daily_demand").mean().alias("avg_daily_demand_plant"),
            pl.col("safety_stock_static").max().alias("safety_stock_static")])
    )

    avg_demand_plant = avg_demand_plant.with_columns([
        (pl.col("avg_daily_demand_plant") +(pl.col("safety_stock_static") / 45)).alias("avg_daily_demand_plant")
    ])

    rows= []
    for row in avg_demand_plant.iter_rows(named=True):
        plant=row["plant_id"]
        item=row["item_id"]
        pallet_size=row["pallet_size"]
        avg_daily=row["avg_daily_demand_plant"]

        cycle_days = rng.choice(cycles, p=cycles_prod[plant])

        ##MOQ
        base_days = rng.choice([ 2, 3, 4], p=[0.6, 0.3, 0.1])
        base_moq = np.ceil(avg_daily * base_days)
        is_challenge = rng.random() < p_challange[plant]
        if is_challenge:
            multiplier = rng.choice([6, 7, 8], p=[0.6, 0.3, 0.1])
        else:
            multiplier= rng.choice([2, 3, 4, 5], p=[0.4, 0.3, 0.2, 0.1])
        moq_row= int(np.round(base_moq * multiplier))
        moq_units = int(np.ceil(moq_row / pallet_size) * pallet_size)

        rows.append({
            "item_id": item,
            "plant_id": plant,
            "avg_daily_demand_plant": avg_daily,
            "cycle_days": int(cycle_days),
            "MOQ_units": moq_units,
        })

    prod_master = pl.DataFrame(rows)
    return prod_master

prod_master = generate_prod_master(demand_outlook)

prod_master.write_csv("Symulation data/prod_master.csv")